<a href="https://colab.research.google.com/github/ashfaq-polit/Large_language_models/blob/master/Agent_supervisor_llama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet -U langchain langchainhub langchain_experimental langchain_huggingface transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [7]:
!pip install --quiet -U langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 20.7 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import os

hf_token = userdata.get("llama3")  # Assumes token is stored in 'llama3'
os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_token,
    torch_dtype=torch.float16,
    device_map="auto"
)

llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    temperature=0.1,
    top_p=0.9,
    return_full_text=False,
    pad_token_id=tokenizer.eos_token_id
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
llm_pipeline = HuggingFacePipeline(pipeline=llama_pipeline)
llm = ChatHuggingFace(llm=llm_pipeline)


In [8]:
from langchain_core.tools import tool
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
import operator
import functools
from typing import Annotated, Sequence, TypedDict

# --- Tools ---
@tool("lower_case", return_direct=False)
def to_lower_case(input: str) -> str:
    """Returns the input as all lower case."""
    return input.lower()

@tool("random_number", return_direct=False)
def random_number_maker(input: str) -> str:
    """Returns a random number between 0-100. input the word 'random'"""
    import random
    return str(random.randint(0, 100))

python_repl_tool = PythonREPLTool()
tools = [to_lower_case, random_number_maker, python_repl_tool]

# --- Agent Creation Function ---
def create_agent(llm, tools, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    agent = create_tool_calling_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=True)

# --- Agent Node Executor ---
def agent_node(state, agent, name):
    current_messages = state.get("messages", [])
    try:
        agent_output = agent.invoke({"messages": current_messages})
        output = agent_output.get("output", str(agent_output))
        return {"messages": [HumanMessage(content=output, name=name)]}
    except Exception as e:
        return {"messages": [HumanMessage(content=f"{name} error: {e}", name=name)]}

# --- Supervisor Agent ---
members = ["Lotto_Manager", "Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the user request, respond with"
    " the worker to act next. When finished, respond with FINISH."
)
options = ["FINISH"] + members
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "type": "object",
        "properties": {
            "next": {"anyOf": [{"enum": options}]},
        },
        "required": ["next"],
    },
}
supervisor_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages"),
    ("system", "Given the conversation, who should act next? {options}"),
]).partial(options=str(options), members=", ".join(members))
supervisor_chain = (
    supervisor_prompt
    | llm.bind_tools(tools=[function_def], tool_choice="auto")
    | JsonOutputFunctionsParser()
)

# --- Graph State ---
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

# --- Agents ---
lotto_agent = create_agent(llm, tools, "You are a lotto manager. Generate random numbers.")
lotto_node = functools.partial(agent_node, agent=lotto_agent, name="Lotto_Manager")

code_agent = create_agent(llm, [python_repl_tool], "You generate Python code to analyze data and plot charts.")
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

# --- LangGraph Setup ---
workflow = StateGraph(AgentState)
workflow.add_node("Lotto_Manager", lotto_node)
workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_chain)

for member in members:
    workflow.add_edge(member, "supervisor")
workflow.add_conditional_edges("supervisor", lambda x: x["next"], {k: k for k in members} | {"FINISH": END})
workflow.set_entry_point("supervisor")

# --- Compile Graph ---
graph = workflow.compile()


In [10]:
config = {"recursion_limit": 20}
final_response = graph.invoke(
    {
        "messages": [
            HumanMessage(content="Get 10 random lotto numbers and plot them on a histogram in 10 bins and tell me what the 10 numbers are at the end")
        ]
    }, config=config
)

ValueError: Last message must be a HumanMessage!